In [1]:
# Basic Dependencies
import nfl_data_py as nfl
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Import hugging face transformers
from transformers import BertTokenizer, BertModel, BertForMaskedLM
from transformers import pipeline